# Recipe Generation using Transformers

Transformers have been shown to outperform custom architectures on many NLP tasks. In text generation particularly, they are able to better capture complex long-range dependencies and generate fluent sentences. However, effective language generation systems must not only create grammatically accurate text but text that is topically consistent and globally coherent. 

Recipe generation is an example of a task where such global coherence is needed. Recipes are essentially a sequence of instructions about how to use a given set of ingredients to create a particular dish. Any recipe generation model needs to keep track of which ingredients from a particular recipe has already been mentioned in the generated text and those which still need to be used. Previous solutions to the global coherence problem have used architectures which combine RNNs like LSTMs and GRUs with a learnt component which tracks the *agenda*, a list of strings which should be mentioned in the output. Inspired by the improved performance of transformers on domain-specific downstream tasks we ask: Can a transformer learn to generate globally coherent text according to a pre-specified agenda?

To answer this, we focus on the simpler task of recipe completion. Given a recipe title, list of ingredients and a fixed span of text from a recipe, we study the performance of a Neural Checklist model and T5 transformer on generating the remaining portion of the recipe.

### How to run this file:

This interactive notebook came as part of a `demo.zip` file also containing some other scripts. If you are running this code on Google Colab, and the unzipped file contents are stored on Google Drive, run the below cell to mount your Drive. After that in the place of `...` in the third line, please indicate the path to the folder containing all the file contents.

> IMPORTANT! Make sure Colab uses a GPU runtime. This can be done by doing Runtime > Change Runtime Type > Set "Hardware Accelerator" to "GPU".

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd ...

## Import Libraries

In [ ]:
# Import packages
from helpers.T5 import *
from helpers.checklist import *
import os, glob, argparse, logging
import pickle, json
import time, random
import re
from tqdm import tqdm
from itertools import chain
from string import punctuation
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Presents our results clearly
pd.set_option('display.max_colwidth', None)

## Some helper functions

In [ ]:
def set_seed(seed):
    """ Ensures reproducibility of our results """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)

def present_recipes(results):
    """ Presents the recipe in a clean DataFrame format """
    gen = list(results['Generated'])
    target = list(results['Target'])
    df = {"Generated": gen, "Target":target}
    df_2 = pd.DataFrame(df)
    return df_2

## Baseline - Neural Checklist 

The neural checklist model was developed by Kiddon et al 2016 for globally coherent recipe generation. Specifically, the model inputs a goal/recipe title, the agenda/list of ingredients and an initial word token, using which it generates the next word. We have adapted the model for our recipe completion case (details are in our report). The below code implements our final trained model on a demo dataset of 10 recipes. 80% of the sentences have been masked out and the model has to generate this remaining portion.  

First, we load in two important files: 

1. `data/test.csv` : Contains 10 demo recipe titles, ingredients and recipe texts.
2. `word2idx.json` : Tokenizer dictionary.

In [ ]:
# Loads the demo data
demo_df = pd.read_csv("data/test.csv")
goals = demo_df['name'] # goals
agendas = demo_df['ingredients'] # agendas

# load word2idx tokenizer
file = open("word2idx.json", "r")
word2idx = json.load(file)
file.close()

# Converts idx back to words
idx2word = {v:k for k,v in word2idx.items()}

### Run the trained model

The below function generates a completed recipe given the `goals` and the `agendas`.

In [ ]:
# Save results here
checklist_results_df = {"Generated":[], "Target":list(demo_df['instructions'])}

# set the goal and agendas
for i in range(len(goals)):
    g = goals[i]
    E = eval(agendas[i])
    start_sent, _ = mask_sentence(demo_df['instructions'][i], mask=0.8)
    start_sent = 'SOS ' + start_sent
    
    g = tokenize_sentence(g, word2idx=word2idx)
    E = tokenize_sentence(E, word2idx=word2idx)
    start_sent_tokenized = tokenize_sentence(start_sent, word2idx)
    
    goal= torch.Tensor(g).type(torch.LongTensor).to(device).reshape(1, -1)
    ingr = torch.Tensor(E).type(torch.LongTensor).to(device).reshape(1, -1)
    start_tokens = torch.Tensor(start_sent_tokenized).type(torch.LongTensor).to(device).reshape(1, -1)
    
    out = beam_search(g=goal, E=ingr, start_tokens=start_tokens)
    checklist_results_df['Generated'].append(tokens_to_sent(out))

### Results

The results of the Neural Checklist model are presented below. We first show the recipe titles and the set of ingredients for this recipe. The output is the remaining portion of the recipe that the model learns to generate. This is contrasted with the target remaining portion of the recipe in the same table. 

In [ ]:
# print generated sentences
demo_df[['name', 'ingredients']]

In [ ]:
present_recipes(checklist_results_df)

## T5 Transformer

The T5 transformer was developed by Raffel et al 2020 for 'text-to-text' framework tasks. An example of such a task is for example, we provide a model with the text "Translate English to German: That is good!" and the model predicts "Das ist gut!" as the translation. The original model has been trained on a large Common Crawl Corpus and is already provided as part of the HuggingFace Library. We fine-tune it for the recipe completion case. At test time, we provide the model with a text like "*Recipe title*, *Ingredients*, *Initial text* and using this, the model will generate the remaining portion of the recipe.

The class below contains the essential functions to preprocess and prepare our recipe dataset for fine-tuning the T5 model.

In [ ]:
class RecipeDataset(Dataset):
    
    def __init__(self, tokenizer, data_dir, type_path,  max_len=512, mask_percent=0.2, mode='fill'):
        self.data_dir = data_dir
        self.type_path = type_path
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.ingredients = []
        self.names = []
        self.mode = mode
        self.mask_percent = mask_percent
        self.proc = RecipeProcessor()
        self._build()
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, 
                "target_ids": target_ids, "target_mask": target_mask}
  
    def __len__(self):
        return len(self.inputs)
  
    def _build(self):
        if self.type_path == 'train':
            examples = self.proc.get_train_examples(self.data_dir)
        elif self.type_path == 'val':
            examples = self.proc.get_dev_examples(self.data_dir)
        elif self.type_path == 'test':
            examples = self.proc.get_test_examples(self.data_dir)

        for i, example in enumerate(examples):
            if i % 10000 == 0:
                print(i)
            self._create_features(example)
        
    def _create_completion(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          input_words = []
          label = []
        else:
          mask_words = np.round(self.mask_percent*total_words).astype(int)
          input_words = words[:mask_words] + ['<extra_id_0>']
          label = ['<extra_id_0>'] + words[mask_words:] + ['<extra_id_1>']

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
    
    def _create_features(self, example):
    
        ingredients_ = ",".join(example.ingredients)
        name_ = example.name
        
        if self.mode == 'fill':
          masked_, label_ = self._create_masks(example.steps)
        
        elif self.mode == 'complete':
          masked_, label_ = self._create_completion(example.steps)

        elif self.mode == 'shuffle_complete':
          masked_, label_ = self._create_completion(example.steps)  
                                        
        input_ = "name: %s  ingredients: %s masked: %s </s>" % (name_, ingredients_, masked_)
        target =  label_ + " </s>"
        
        # tokenize inputs
        tokenized_inputs = self.tokenizer.batch_encode_plus(
            [input_], max_length = 256, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )
        # tokenize targets
        tokenized_targets = self.tokenizer.batch_encode_plus(
            [target], max_length = 128, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )

        self.inputs.append(tokenized_inputs)
        self.targets.append(tokenized_targets)
        self.ingredients.append(example.ingredients)
        self.names.append(example.name)

### Run the fine-tuned T5

The following function `generate_recipes` sets up our already fine-tuned best T5 model and uses it to generate predictions on sample data.

In [ ]:
def generate_recipes(data_dir=None, checkpt_fpath=None, mode = 'fill'):
    
    # Load T5 fine-tuning model with fine-tuned weights
    model = T5FineTuner.load_from_checkpoint(checkpt_fpath)

    # Load T5 tokenizer
    tokenizer = T5Tokenizer.from_pretrained("t5-base")

    # Load Recipe dataset
    dataset =  RecipeDataset(tokenizer, data_dir=data_dir, type_path='test', mode=mode)
    loader = DataLoader(dataset, batch_size=len(dataset), num_workers=2)

    model.model.eval()
    outputs = []
    targets = []

    for i, batch in enumerate(tqdm(loader)):

        # Need to change max. length argument
        outs = model.model.generate(input_ids=batch['source_ids'], 
                                    attention_mask=batch['source_mask'], 
                                    max_length=128)
        dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
        target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

        # Store the generated sentences
        outputs.extend(dec)
        targets.extend(target)
    
    # Store results
    results = {'Recipe Title':dataset.names, 'Ingredients List':dataset.ingredients, 'Generated':outputs, 'Target':targets}
  
    return results

In [ ]:
t5_results = generate_recipes(data_dir="data/", checkpt_fpath="weights/exp_10_rc_noise_ef.ckpt", mode="shuffle_complete")

### Results

Like in the Neural checklist, we again present the results first by giving the set of original recipe titles and ingredients. This is followed by a table contrasting the generated and target recipe completions.

In [ ]:
# print generated sentences
t5_results_df = pd.DataFrame(t5_results)
t5_results_df[['Recipe Title', 'Ingredients List']].head(10)

In [ ]:
present_recipes(t5_results_df)